<style>
.jp-Notebook {
    margin: auto;
    max-width: 1024px;
}
.jp-InputPrompt {
    display: none;
}
</style>

# SARSA

SARSA is a reinforcement learning algorithm. The name describes the algorithm: __S__tate __A__ction __R__eward __S'__tate __A'__ction.

In its simplest form:
- given a State _s_
- refer to the policy and select an Action _a_
- Loop
    - observe the Reward _r_ for the selected action
    - observe the State _s'_ given the current state _s_ and selected action _a_
    - refer to the policy and select the next Action _a'_
    - update Q(s, a) based on the observed reward and the selected states and actions
    - repeat with s <- s', a <- a'

In this report we will:

1. Describe the problem statement
2. Discuss how to implement SARSA given the problem statement, and illustrate with code snippets
3. Explore the results, and measure performance and the effects of changing various parameters of the algorithm
4. Cover on-policy (SARSA) as well as off-policy (Q-Learning) algorithms

Our implementation will use an eligibility trace $TD(\lambda)$ to speed up learning, and our policy will be $\epsilon$-greedy. We will approximate the state in the continuous space, and model the experiment with a neural network.

Libraries used: __numpy__ and __matplotlib__.

---




## Problem Statement


### Geography of the experiment
A point-like agent is moving in a continuous 2d state space.
  
The 2d space is of unit length.

A 20 by 20 grid of place cells, distributed uniformly, will be used to approximate the state given the agent's coordinates.

A trial starts with the agent at $s_0=[0.1,\ 0.1]$.

At each step, the agent moves a distance of $l=0.03$ in one of 8 directions.

A goal area is defined as a circle of radius 0.1 around the centre with coordinates $[0.8,\ 0.8]$ 

![](./images/geometry_of_experiment.svg '') ![](./images/firstMoves.svg '')



### Policy
The direction is selected according to an $\epsilon$-greedy policy:

$$
\begin{equation}
 P= \left\{
        \begin{array}{ll}
            \epsilon & \quad random\ a' \\
            1- \epsilon & \quad \max_{a'}Q(s, a')
        \end{array}
    \right.
\end{equation}
$$

A random direction is taken with probability $\epsilon$ and the direction with the highest learnt reward will be selected with probability $1 - \epsilon$.



### Rewards
$R=-2$ : If the agent's coordinates $s=[s_x,\ s_y]$ are outside the grid after taking an action _a_, return the agent to the grid.

$R=10$: If the agent is within 0.1 units from $[0.8,\ 0.8]$.

$R=0$: Any other action gets $R=0$.


### Termination
A trial stops either when the agent reaches the goal or after $N_{MAX}=10000$ steps.


### Function approximation
Approximate the state. Each cell has an activation depending on the agent's current coordinates: 

$$
\begin{equation}
  h_j(s)=\exp({-\frac{(x_j-s_x)^2+(y_j-s_y)^2}{2\sigma^2})}
\end{equation}
$$

Each cell is connected to each action by a weight: $w_{aj}$.

The Q function can then be estimated as:

$$
\begin{equation}
  Q(s,a)=\sum_j{w_{aj}h_j(s)}
\end{equation}
$$


### Task

Implement the SARSA and Q-Learning algorithms, and run 50 trials over 10 agents.

Start with $\epsilon=0.5$. Try different values and decaying over trials.

Plot learning curves, total reward, number of steps to reach goal.


### Algorithms

More formal algorithm definitions for Sarsa and Q-Learning, with and without eligibility trace:



>---
>__Algorithm 1:__ _SARSA on-policy temporal difference TD(0)_

>---
> Algorithm parameters: step size  $\alpha \in (0 , 1],$ $\epsilon > 0$   
Initialize  $Q  ( s, a ), \  \forall s \in S^+ , a \in A ( s ),$ arbitrarily
>
> Loop for each episode:  
$\quad$Initialize $S$   
$\quad$Choose $A$ from $S$ using some policy derived from $Q$ (eg $\epsilon$-greedy)  
$\quad$Loop for each step of episode:      
$\qquad$Take action $A$, observe $R,$ $S'$   
$\qquad$Choose $A'$ from $S'$ using the same policy as used to select $A$ (eg $\epsilon$-greedy)   
$\qquad Q(S,A) \leftarrow Q(S, A) + \alpha[R+\gamma $ <span style="background-color:lightgrey; padding: 1px;">$Q(S', A')$</span>$ - Q(S, A)]$   
$\qquad S \leftarrow S'$  
$\qquad A \leftarrow A'$    
$\quad$until $S$ is terminal

>---


>---
>__Algorithm 2:__ _Q-Learning off-policy TD(0)_

>---
> Algorithm parameters: step size  $\alpha \in (0 , 1],$ $\epsilon > 0$   
Initialize  $Q  ( s, a ), \  \forall s \in S^+ , a \in A ( s ),$ arbitrarily   
>
> Loop for each episode:  
$\quad$Initialize $S$   
$\quad$Loop  for  each  step  of  episode:    
$\qquad$Choose  $A$ from $S$ using some policy derived from $Q$ (eg $\epsilon$-greedy)   
$\qquad$Take action $A$, observe $R,$ $S'$   
$\qquad Q(S,A) \leftarrow Q(S, A) + \alpha[R+\gamma$ <span style="background-color:lightgrey; padding: 1px;">$\max_{A'}Q(S', A')$</span>$ - Q(S, A)]$   
$\qquad S \leftarrow S'$    
$\quad$until $S$ is terminal

>---


>---
>__Algorithm 3:__ _SARSA on-policy temporal difference TD($\lambda$); with eligibility trace_

>---
> Algorithm parameters: step size  $\alpha \in (0 , 1],$ $\epsilon > 0$   
> Initialize  $Q  ( s, a ), \  \forall s \in S^+ , a \in A ( s ),$ arbitrarily  
> Initialize $E(s,a), \  \forall s \in S^+ , a \in A ( s ),$ as 0s
>
> Loop for each episode:  
$\quad$Initialize $S$   
$\quad$Choose $A$ from $S$ using some policy derived from $Q$ (eg $\epsilon$-greedy) 
$\quad$Loop for each step of episode:      
$\qquad$Take action $A$, observe $R,$ $S'$   
$\qquad$Choose $A'$ from $S'$ using the same policy as used to select $A$ (eg $\epsilon$-greedy)   
$\qquad \delta = R+\gamma Q(S', A') - Q(S, A) $  
$\qquad E(S,A) \leftarrow E(S, A) + 1$, (note: use function approx increment for continuous space, instead of 1)  
$\qquad \forall s,a:$  
$\quad\qquad Q(s,a) \leftarrow Q(s,a) + \alpha\delta $ <span style="background-color:lightgrey; padding: 1px;">$E(s,a)$</span>  
$\quad\qquad E(s,a) \leftarrow \gamma\lambda E(s,a)$  
$\qquad S \leftarrow S'$  
$\qquad A \leftarrow A'$    
$\quad$until $S$ is terminal

>---


---

## Implementation

Starting by writing code snippets in a jupyter notebook, then a collection of functions and global variables, the final implementation will be class-based. We will touch on debugging, testing and performance.
In this report simplified code snippets are provided. To read the source code and view how the plots were made, refer to the git repository [https://github.com/michaelhobbs/sarsa].


In [1]:
import numpy as np
import math

### Agent
The state is a 2d array of coordinates. $s = [s_x, s_y]$

The agent will need to be able to select a random direction, we wll need to determine **s'** given **s** and **a**, including handling movements at the edge of the grid and detecting when **s'** is in the goal area.

In [ ]:
numDirections = 8
def selectRandomDirection():
	randomDirection = np.random.randint(0, numDirections)
	return randomDirection

l = 0.03
diagonalDelta = math.sqrt(l**2 /2)
positionDeltas = np.array([ \
    [0, -l],  # down \
    [diagonalDelta, -diagonalDelta], # down right \
    [l, 0], # right \
    [diagonalDelta, diagonalDelta], # up right \
    [0, l], # up \
    [-diagonalDelta, diagonalDelta], # up left \
    [-l, 0], # left \
    [-diagonalDelta, -diagonalDelta] ]) # down left

def updatePosition(currentCoords, direction):
    return currentCoords + positionDeltas[direction]

minCoords = [0, 0]
maxCoords = [1, 1]
def detectCollision(coords):
    return coords[0] < minCoords[0] \
        or coords[1] < minCoords[1] \
        or coords[0] > maxCoords[0] \
        or coords[1] > maxCoords[1]

def detectGoal(coords):
	return np.sqrt((coords[0]-0.8)**2 + (coords[1]-0.8)**2) <= 0.1

def returnRatToGrid(coords):
	'''return closest point inside grid to arg coords'''
	return np.minimum(np.maximum(minCoords, coords), maxCoords)

We now have all the building blocks for a single time step using a random selection of actions.

Let's run 100'000 steps with this completely random agent.

We will track rewards and follow the rules when collisions are detected by keeping the agent inside the grid when an action would take it outside.

We will reset to the starting coordinates when the agent reaches the goal area.

In [ ]:
reward = 0
startingPosition = [0.1, 0.1]
currentPosition = startingPosition
history = np.array([currentPosition])
iteration = 0
while iteration < 100000:
    randomDirection = selectRandomDirection()
    newPosition = updatePosition(currentPosition, randomDirection) # s'
    hasCollision = detectCollision(newPosition) # by taking action a given state s -> s'
    hasGoal = detectGoal(newPosition) # by taking action a given state s -> s'
    currentPosition = newPosition
    currentReward = 0
    if hasCollision:
        currentReward = -2
    elif hasGoal:
        currentReward += 10
    reward += currentReward

    history = np.append(history, [currentPosition], axis=0)

    if hasCollision:
        currentPosition = returnRatToGrid(currentPosition)
    if hasGoal:
        currentPosition = startingPosition

    iteration += 1

States visited:

 ![](./images/history.png '') ![](./images/historyHistogram.png '')

Our agent has visited most states with the exception of some in the top-righthand corner of the grid, behind the goal area. This makes sense as the goal area blocks exploration behind it by resetting the agent to the starting point.

### Neural Network

For the neural network we will need to calculate the activation of every neuron. We could do this with for loops, but this would not be very efficient. By vectorizing our neural network (weights, eligibility trace, input layer activation) we can perform faster vector operations with numpy. I decided to flatten all data into 2 dimensional vectors of length 20*20. The first 20 elements correspond to the first 20 cells in the grid along the bottom, the next 20 elements in the vecotrs are the second row of cells, and so on. The weights and eligibility trace have a second dimension: the directions. Each cell in the grid has 8 possible actions, and each of these is modelled by a weight, which converts the place cell activity _h(s)_ to its contribution towards picking that direction.

First we will implement _h(s)_, and plot it to check what it looks like. 

Then we will implement the output neuron activation _Q(s,a)_.

Finally, we will implement the eligibility trace, its update and decay.



#### State approximation

$\begin{equation}
  h_j(s)=\exp({-\frac{(x_j-s_x)^2+(y_j-s_y)^2}{2\sigma^2})}
\end{equation}$

In [ ]:
def calculatePlaceCellActivity(ratCoords, placeCellCoords, sigma = 0.05):
	denom = 2 * sigma**2
	num = -((placeCellCoords[:, 0]-ratCoords[0])**2 + (placeCellCoords[:, 1]-ratCoords[1])**2)
	return np.exp(num / denom)

Place cell activity visualized when agent is in the centre of the grid: $[0.5,\ 0.5]$

![](./images/placeCellActivity.svg '')

Place cell activity visualized when agent is in the starting position: $[0.1,\ 0.1]$

![](./images/placeCellActivityStart.svg '')


The closest neighbours contribute to the decision. The neighbours of the closest neighbours already do not contribute significantly.

#### Output neuron activity

$Q(s,a)=\sum_j{w_{aj}h_j(s)}$

In [ ]:
xAxisSteps = 20
yAxisSteps = 20
weights = 0.001 * np.array(np.random.rand(xAxisSteps * yAxisSteps, numDirections))
# Q(s, a)
def calculateOutputNeuronActivityForDirection(ratCoords, direction, weights):
	return np.sum(np.multiply(weights[:, direction], calculatePlaceCellActivity(ratCoords, placeCellCoords)))

def calculateOutputNeuronActivity(ratCoords, weights):
	return [calculateOutputNeuronActivityForDirection(ratCoords, a, weights) for a in range(numDirections)]

The eligibility trace is first increased by the activation of the input neurons for the selected action. This is the contribution of each place cell towards the decision:
$$
e_a(t) = e_a(t-1) + h(s)
$$

This state of the eligibility trace is used in the weight update. Weights are updated by:
$$\Delta_w = \eta * e_a * (r - Q(s,a) + \gamma * Q(s',a'))$$
Where:
$\eta$: learning rate
$\gamma$: reward discount rate

After updating the weights, the entitre eligibility trace will decay. This prevents actions taken a long time ago from affecting the learning as much as the actions which just occurred : 
$$
e_a(t) = \lambda * \gamma * (e_a(t-1) + h(s))
$$
Where:
$\lambda$: eligibility trace decay rate

In [ ]:
eligibilityTrace = np.zeros((xAxisSteps * yAxisSteps, numDirections)) # at start of experiment there is no trace

def updateEligibilityTrace(currentCoords, direction, eligibilityTrace):
	return eligibilityTrace[:, direction] + calculatePlaceCellActivity(currentCoords, placeCellCoords)

def updateWeights(delta, weights, eligibilityTrace):
	return weights + (learningRate * delta * eligibilityTrace)

def decayEligibilityTrace(eligibilityTrace):
	return eligibilityTrace * rewardDiscountRate * eligibilityTraceDecayRate

### Single Step

Here we simply put everything together in a single function. We will adapt and extend thet code snippet we wrote to test the grid setup, where we ran a randomly acting agent for 100'000 time steps.

The only part missing is the $\epsilon$-greedy policy.



In [ ]:
def selectDirection(epsilon, ratCoords, weights):
	randomNumber = np.random.rand(1)
	if (randomNumber < epsilon):
		return selectRandomDirection()
	else:
		outputNeuronActivity = calculateOutputNeuronActivity(ratCoords, weights)
		arr = np.array(outputNeuronActivity)
		maxElement = np.amax(arr)
		maxIndexes = np.where(arr == maxElement)
		randomFromMaxIndexes = np.random.choice(maxIndexes[0]) # pick random index when several share maxValue
		return randomFromMaxIndexes

def runSingleTimeStep(currentPosition, currentDirection, weights, eligibilityTrace, epsilon):
	reward = 0
	newPosition = updatePosition(currentPosition, currentDirection) # s'
	hasCollision = detectCollision(newPosition) # by taking action a given state s -> s'
	hasGoal = detectGoal(newPosition) # by taking action a given state s -> s'
	if (hasCollision):
		newPosition = returnRatToGrid(newPosition)
		reward -= 2
	elif (hasGoal):
		reward += 10
	# update eligibility trace at ratCoords before move for chosen direction
	eligibilityTrace = updateEligibilityTrace(currentPosition, currentDirection, eligibilityTrace)

	# calculate next action a' which will be taken in next iteration and will be used to update the model
	newDirection = selectDirection(epsilon, newPosition, weights)
	anticipatedOutputNeuronActivityInNextStep = calculateOutputNeuronActivityForDirection(newPosition, newDirection, weights)

	delta = reward - calculateOutputNeuronActivityForDirection(currentPosition, currentDirection, weights) \
         + rewardDiscountRate * anticipatedOutputNeuronActivityInNextStep
	weights = updateWeights(delta, weights, eligibilityTrace)
	eligibilityTrace = decayEligibilityTrace(eligibilityTrace)
	return [newPosition, newDirection, reward, hasGoal, weights, eligibilityTrace]

### Trial

A trial takes an agentand runs time steps until reaching an end condition. As a reminder, end conditions are running N_max steps without the agent finding the goal area, or the agent entering a state within the goal area.

We will wrap the single time step function in a loop until a trial termination condition is met. You will notice some print and file io operations, this is to be able to see more or less how far along the execution of the episode is while it is running, and saves data for analysis later. It is useful to save as much data as possible because the execution may take a lot of time, and it is very inefficient to have to re-run the entire experiment to collect more data later.


In [ ]:
def runTrial(weights, eligibilityTrace, epsilon, trialNumber, experimentNumber):
	ratCoords = [0.1, 0.1]
	currentDirection = selectDirection(epsilon, ratCoords, weights) # initial action a
	trialReward = 0
	maxSteps = 10000
	currentStep = 0
	hasGoal = False

	while (currentStep < maxSteps and not hasGoal):
		[ratCoords, currentDirection, reward, hasGoal, weights, eligibilityTrace] = \
			 runSingleTimeStep(ratCoords, currentDirection, weights, eligibilityTrace, epsilon)
		# track coords for analysis
		trialReward += reward
		currentStep += 1

	# save all coords for analysis
	return [weights, eligibilityTrace, hasGoal, currentStep, trialReward, ratCoords]

### Experiment

Our experiment will consist of 50 trials per agent.

We will wrap the `runTrial` function in an `runExperiment` function. It will be useful to save the weights (Q-values) between trials to see how they evolve. Tracking trial reward is also interesting to measure progress. Print and file operations have been omitted in the report but can be found in the source code in github.

In [ ]:
def runExperiment(experimentNumber):
    maxTrials = 50
    currentTrial = 0
    epsilon = 0.9
    weights = 0.001 * \
        np.array(np.random.rand(xAxisSteps * yAxisSteps, numDirections))
    while (currentTrial < maxTrials):
        eligibilityTrace = np.zeros((xAxisSteps * yAxisSteps, numDirections)) # reset eligibility trace
        [weights, eligibilityTrace, hasGoal, currentStep, trialReward, ratCoords] = runTrial(
            weights, eligibilityTrace, epsilon, currentTrial, experimentNumber)
        # optionally update epsilon
        # save weights for analysis
        currentTrial += 1

### Debugging and Tests

In order to write tests your functions have to be importable. The easiest way to do this is to separate your main script (the one you run) and your functions. Then you can import your functions in your test file and write unit tests.

Refer to the source code in github to see some tests for detecting collisions and goal states, as well as place cell activity and Q value calculation.


---

## Results

### Theoretical best
Euclidean distance between starting point and centre of goal area, minus the radius of the goal area.

$\sqrt{(0.8-0.1)^2+(0.8-0.1)^2}-0.1 = 0.8899$

Distance to closest point of goal area by movement speed:

$\frac{0.8899}{0.03} = 29.66$

The shortest path to the goal consists of 30 steps.

### On- vs. Off- Policy
Sarsa is very similar to Q-Learning. The only difference between the two algorithms is that while Sarsa uses the action selected for the weight update as the next action the agent will take, Q-Learning updates the weights using the best action available at the next time, given its current understanding of the \[state, action\] space. You will find the update equations below:

__Sarsa__
$\Delta_w = \eta * e_a * (r - Q(s,a) + \gamma * Q(s',a'))$

__QL__
$
\begin{equation}
\Delta_w = \eta * e_a * (r - Q(s,a) + \gamma * Q_{max_{a'}}(s',a'))
\end{equation}
$

In general Q-Learning's approach promotes exploration, however, when using a funcation approximation for the state it is not guaranteed to converge. This is because the behaviour policy and the target policy are different (Baird 1995). i.e. the action used in the update rule is different to the action actually taken in the next step.

In our example experiment, off-policy learning fails to converge. The weights (Q-values) explode and eventually become `NaN`s. More elaborate algorithms have been proposed to ensure convergence while still addressing the curse of dimensionality with continuous state and very large state sets.

The take-away here is that not all learning algorithms can be applied to the continuous state space, and the algorithm to use needs to be selected depending on the problem statement.


### Exploration vs. Exploitation

In order to learn the approximate future rewards (Q-values) for each action in each state, the agent should visit and take these actions. Since the future reward depends on all future actions, the number of possible paths to learn and to take into account when estimating the future reward grows exponentially.

The $\epsilon$ parameter allows us to control how often the agent takes a random direction. In this way, the agent can explore the state space, and better estimate the future rewards. This parameter also allows the agent to break out of sub-optimal paths, by sometimes moving randomly the agent may find shorter paths.

In this report we compare various strategies for managing the value of $\epsilon$:
- **constant value**. We tried values in \[0, 0.1, 0.25, 0.5, 0.75, 0.9, 1\]
- **linear decay** from 1 to 0. Decreases the same amount at each new trial until reaching the minimum of 0.
- **logistic decay** following: $log(10-(10-1)*(t/49))$
- **exponential decay** following: $exp(-t/exp(1))$

Visually, it is easier to understand how these strategies will affect learning.

![](./images/linDecay.svg '')![](./images/logDecay.svg '')![](./images/expDecay.svg '')

**Exponential decay** will have a short exploration phase.

**Linear decay** will give equal weight to exploration and exploitation. Starting by heavily exploring the actions, it will gradually enter an exploitation phase.

**Logistic decay** will prioritize exploration, and only start eploitation in the final couple of trials.

#### Performance
Plotted below are the mean rewards observed in each of the trial runs of the experiment. This has been averaged out over 10 agents. Each line is using a different value/strategy of $\epsilon$.

![](./images/avgReward.svg '')
![](./images/averageNumSteps.svg '')

From this first glance into the results, we observe that the completely random agent ($\epsilon$=0). The random agent has a high trial duration and very low reward. The opposite agent ($\epsilon$=1), also has a very high trial duration at the end of the experiment, however the agent learnt to avoid the negative rewards and does not have a low average reward. This agent sometimes gets stuck in sub-optimal loops and never explores outside them.

Zooming into the last 10 trials and only plotting the strategies which performed well, we get the following plots:

![](./images/exploitingReward.svg '')
![](./images/exploitingDuration.svg '')

Two well performing $\epsilon$ strategies failed to cinsistently achieve the maximum reward: constant $\epsilon$ values 0.75 and 0.5. In addition to not achieving top rewards, their trial duration was higher than the best scoring strategies and were not included in the zoom-in plot of the trial durations.

The average duration plot shows that the exponential decay strategy suffered from having a shorter exploration period. The constant value strategies were beaten by the linear and logistic decay strategies, with the agent taking a random direction 10% of the time performing better than the 25% random agent.

Finally, the linear agent performed best until the very end, when the logistic agent entered the fully exploitation phase. The logistic decay strategy resulted in the shortest path (30 steps) being discovered by most of the 10 agents who ran the experiment.

Table of final performance measured as average reward and duration of trials taken over the final 5 trials.

| Strategy      | Reward           | Duration  |
| :------------ |:----------------:| ---------:|
| 0             | 6.9              | 4024.6    |
| 0.1           | 9.2              | 45.0      |
| 0.25          | 7.8              | 56.1      |
| 0.5           | 7.2              | 96.7      |
| 0.75          | 7.6              | 152.8     |
| 0.9           | -1.4             | 453.2     |
| 1             | -240.1           | 2573      |
| linear        | -3.7             | 36.2      |
| logistic      | -7.3             | 35.7      |
| exponential   | 4.76             | 53.8      |


Table of final performance measured as average reward and duration of trials taken over the final trial.


| Strategy      | Reward           | Duration  |
| :------------ |:----------------:| ---------:|
| 0             | 6                | 4024.6    |
| 0.1           | 10               | 44.4      |
| 0.25          | 10               | 57.4      |
| 0.5           | 10               | 90.2      |
| 0.75          | 8.6              | 156.1     |
| 0.9           | -2.8             | 533.7     |
| 1             | -245.8           | 2519.8    |
| linear        | 10               | 34.7      |
| logistic      | 10               | 30.8      |
| exponential   | 10               | 53.2      |


These results demonstrate the need for a balance between exploration (high $\epsilon$) and exploitation (low $\epsilon$). One can imagine a good approach being to initially set a high value to $\epsilon$, and as the agent discovers and learns the (state, action)-space, decrease it. One could also imagine approaches where this paremeter would be hand-tuned over a period of days, or maybe modifying automatically following some rules on the observed reward. eg: if a trial results in a low reward, increase $\epsilon$ and enter an exploration phase; and if a trial results in a high reward, descrease $\epsilon$ (enter exploitation phase).

### Elegibility Trace

Learning with an eligibility trace means that Q-value updates are applied to the previous \[state, action\] pairs visited. Without an eligibility trace, the only Q-value updated is the \[state, action\] pair which triggered the reward. As a result, the agent must run paths multiple times in order for reward information to trickle back to the earlier states in the path to a reward state.

As we are using function approximation, our Q-value updates will already affect a neighbourhood of states, even without the use of an eligibility trace. Since our problem is very simple and when a reward is observed for an action, it is valid for almost all states using that same action, the eligibility trace speed up is not very prominent. In fact we only observe the speed up in the first few trials.

![](./images/durationWithElig.svg '')
![](./images/durationWithoutElig.svg '')





### Evolution of weights
It can help to visualise the evolution of the Q-values, our weights. Here we will demonstrate one way these can be visualised.

**Quiver plot**
For each place cell we will find which action has the largest Q-value, and we will plot an arrow centred at the place cell coordinates in the direction corresponding to the largest Q-value. We illustrate with quiver plots generated before and after training. $\epsilon = 0.5$

![](./images/quiver/multi.svg '')

### Weight initialization
To gain insight in how important weight initialization is for Sarsa, the experiment has been run with all other parameters fixed, $\epsilon = 0.5$, using three weight initialization strategies, as always averaged over 10 agents:
- Small random values in \[0, 0.001\]
- Zeros
- Random in \[0, 1\]

![](./images/weightInitRewards.svg '') ![](./images/weightInitSteps.svg '')

One agent failed to find a path to the goal for the first 40 trials when weights were initialized using very small random values. This heavily penalises that strategy when comparing to the others. The zero weight initialization seems to have performed best. All the same, there is no strong winner, and the low values initialization may have simply been unlucky. Were we to run using 100 agents instead of only 10, perhaps we would see the other strategies also having agents which fail to find the goal.


---

## Conclusions

We saw the trade off between exploration and exploitation in reinforcement learning. An agent with no prior knowledge of the environment needs to explore it before being able to exploit its understanding of the environment. In Sarsa, this can be controlled by the value of $\epsilon$ when using an $\epsilon$-greedy policy.

We saw how to use function approximation to model continuous state spaces, and discovered that Q-Learning (off-policy) will not always converge when using function approximation.

We showed how an eligibility trace can be used to speed up propagation of observed rewards, speeding up learning, and saw that measuring performance using only 10 agents can suffer from "noise". eg: if one of the 10 agents performs badly, due to randomness in the action selection, then the average performance measured suffers greatly. We proposed a way to mitigate this: running over more agents averaging out a performance measurement.

We considered 3 weight initialization strategies and found that initializing weights to 0s seems to be a valid choice, at least in our environment.

An implementation was proposed, using vectorization for array operations and we generated human-readable representations of the learning process.

## Open questions / Future tasks / Possible extensions

How should we manage $\epsilon$ over time were the grid layout to change? e.g. the goal area moves every 20 episodes.

How would we tackle the problem of a "shadowed" optimal goal? We saw that our agent barely ever reaches behind the goal area. If a second goal area with a much higher reward were put in the top righthand corner of the grid, how should we set our parameters to increase the likelihood of discovery of the path to the highest paying goal area?

Could a deep neural network improve performance?
Fitted Q-Iteration (FQI); Experience Replay in Deep Q-networks (DQN); Least-square temporal difference (LSTD)

Could we add a second agent, a hunter, whose target is to catch the first agent? (positive reward when in the same cell as the first agent). How would we have to tweak the parameters? How many states would we have and how would we adapt our function approximation?

Can we improve performance and speed up our implementation? Profile the code to find the slowest parts of the implementation. (hint: try caching Q-values and function approx)

Can we build a maze and test our learning algorithm on a more complicated environment? How long would agents need to learn such an environment?  
_See `maze.py` and `maze_` prefixed files under `images` for an example of how this may be implemented and to get an idea of how this performs (esp. images/animations/mega_maze/maze_50_17.png)_

### Bonus:

Videos of final trial of learning process for different epsilon strategies.

<video controls="controls" poster="./images/animations/last_trial_0.1.png">
  <source type="video/mp4" src="./images/animations/last_trial_0.1.mp4"></source>
  <p>Your browser does not support the video element.</p>
</video>
<video controls="controls" poster="./images/animations/last_trial_0.5.png">
  <source type="video/mp4" src="./images/animations/last_trial_0.5.mp4"></source>
  <p>Your browser does not support the video element.</p>
</video>
<video controls="controls" poster="./images/animations/last_trial_0.9.png">
  <source type="video/mp4" src="./images/animations/last_trial_0.9.mp4"></source>
  <p>Your browser does not support the video element.</p>
</video>
<video controls="controls" poster="./images/animations/last_trial_expDecay.png">
  <source type="video/mp4" src="./images/animations/last_trial_expDecay.mp4"></source>
  <p>Your browser does not support the video element.</p>
</video>
<video controls="controls" poster="./images/animations/last_trial_linearDecay1-0.png">
  <source type="video/mp4" src="./images/animations/last_trial_linearDecay1-0.mp4"></source>
  <p>Your browser does not support the video element.</p>
</video>
<video controls="controls" poster="./images/animations/last_trial_logDecay.png">
  <source type="video/mp4" src="./images/animations/last_trial_logDecay.mp4"></source>
  <p>Your browser does not support the video element.</p>
</video>


### Completely random agent when epsilon set to 0

The agent trained with epsilon = 1 spent the entire experiment in exploration mode and I expected it to perform best. However, the Quiver plot of argmax(Q) and running an extra trial using the Q-values of a random agent, but this time setting it to only exploit ($\epsilon=0$), show that this agent failed to find the shortest path to the goal. It managed to have all edge states pointing into the grid, but the rest looks almost random. I suppose this is due to the eligibility trace always being full of random actions when a goal state is reached.

![](./images/exploit/quiver.svg '')

<video controls="controls" poster="./images/animations/explorer.png">
  <source type="video/mp4" src="./images/animations/explorer.mp4"></source>
  <p>Your browser does not support the video element.</p>
</video>

